In [ ]:
# We'll first import all relevant libraries that we will require to access a website's HTML and extract information from the same.

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Using Beautifulsoup for scraping the data
# Beautiful Soup is a Python package for parsing HTML and XML documents.It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping
# Using request library for making HTTP/1.1 requests using Python 

In [ ]:
# Initially, I defined the function get_soup_parsed_tree, that accepts a url, type of HTML tag, class of respective tag and uses BeautifulSoup library to get the HTML for a webpage.



In [1]:
def get_soup_parsed_tree(link, tag_type, div_class):
    website_url = requests.get(link).text
    soup = BeautifulSoup(website_url,'lxml')
    My_table = soup.find(tag_type,{'class':div_class})
    return My_table

In [ ]:
# The webpage includes the information we need in the form of HTML table. Thus, we need to reach that table and extract the information. However, there might be multiple tables on the page. We would thus need to find the class of that table and then access its data.
# 'wikitable sortable' is required class for our table

In [2]:
main_table = get_soup_parsed_tree('https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population', 'table', 'wikitable sortable')

In [3]:
main_table

<table class="wikitable sortable" style="text-align:center">
<tbody><tr>
<th>2018<br/>rank
</th>
<th>City
</th>
<th>State<sup class="reference" id="cite_ref-5"><a href="#cite_note-5">[c]</a></sup>
</th>
<th>2018<br/>estimate
</th>
<th>2010<br/>Census
</th>
<th>Change
</th>
<th colspan="2">2016 land area
</th>
<th colspan="2">2016 population density
</th>
<th>Location
</th></tr>
<tr>
<td>1
</td>
<td style="text-align:left;background-color:#cfecec"><i><a href="/wiki/New_York_City" title="New York City">New York City</a></i><sup class="reference" id="cite_ref-6"><a href="#cite_note-6">[d]</a></sup>
</td>
<td align="left"><span class="flagicon"><img alt="" class="thumbborder" data-file-height="450" data-file-width="900" decoding="async" height="12" src="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/23px-Flag_of_New_York.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/1/1a/Flag_of_New_York.svg/35px-Flag_of_New_York.svg.png 1.5x, //upload.wikimedia.

In [ ]:
#Next, we explore the each table row, and table data for each city from table.
#Here, I am creating List of list for each city with their given data and Wiki link of each city.

In [4]:
table_body = main_table.find('tbody')
data = []
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    cols.append('https://en.wikipedia.org' + row.find('a', href=True)['href'])
    data.append([ele for ele in cols if ele])

In [5]:
data

[['https://en.wikipedia.org#cite_note-5'],
 ['1',
  'New York City[d]',
  'New York',
  '8,398,748',
  '8,175,133',
  '+2.74%',
  '301.5\xa0sq\xa0mi',
  '780.9\xa0km2',
  '28,317/sq\xa0mi',
  '10,933/km2',
  '40°39′49″N 73°56′19″W\ufeff / \ufeff40.6635°N 73.9387°W\ufeff / 40.6635; -73.9387\ufeff (1 New York City)',
  'https://en.wikipedia.org/wiki/New_York_City'],
 ['2',
  'Los Angeles',
  'California',
  '3,990,456',
  '3,792,621',
  '+5.22%',
  '468.7\xa0sq\xa0mi',
  '1,213.9\xa0km2',
  '8,484/sq\xa0mi',
  '3,276/km2',
  '34°01′10″N 118°24′39″W\ufeff / \ufeff34.0194°N 118.4108°W\ufeff / 34.0194; -118.4108\ufeff (2 Los Angeles)',
  'https://en.wikipedia.org/wiki/Los_Angeles'],
 ['3',
  'Chicago',
  'Illinois',
  '2,705,994',
  '2,695,598',
  '+0.39%',
  '227.3\xa0sq\xa0mi',
  '588.7\xa0km2',
  '11,900/sq\xa0mi',
  '4,600/km2',
  '41°50′15″N 87°40′54″W\ufeff / \ufeff41.8376°N 87.6818°W\ufeff / 41.8376; -87.6818\ufeff (3 Chicago)',
  'https://en.wikipedia.org/wiki/Chicago'],
 ['4',
  'H

In [ ]:
# get_demonym function is used for geting information which is beyond the initial table. It will get the data from individual city pages. I have used 'Demonym' as an extra data from each city while scraping the data
# data_cleasing function is used for cleaning the data. E.g.Density of particular city will always in Integer, but while scapring the data it will be text. Hence for cleaning such type of data, I have build this function.
# Also, It will clean the data in the format which can handled by the big query tables.
# Finally, I am creating data frames for each columns of data, which I am using for creating the CSV file.

In [9]:
def get_demonym(city_link):
    city_info = get_soup_parsed_tree(city_link, 'table', 'infobox geography vcard')
    demonym = ''
    for tr in city_info.find('tbody').find_all('tr'):
        if (tr.get('class') == ['mergedtoprow'] or tr.get('class') == ['mergedrow']):
            link = tr.find('a', title='Demonym')
            if link: 
                demonym = tr.find('td').get_text().strip('\n')
    return demonym

def data_cleansing(list,type):
    for ele in list:
        elePos = list.index(ele)
        new_element = type(ele.replace('/', ' ').replace(',', '').split()[0])
        list[elePos] = new_element
    return list

df = pd.DataFrame()
rank = []
city = []
state = []
estimate_2018 = []
census_2010 = []
change = []
land_area_sqmi_2016 = []
land_area_sqkm_2016 = []
population_density_sqmi_2016 = []
population_density_sqkm_2016 = []
location = []
city_link = []
demonyms = []

for i in range(1,len(data)):
    rank.append(data[i][0])
    city.append(data[i][1])
    state.append(data[i][2])
    estimate_2018.append(data[i][3])
    census_2010.append(data[i][4])
    change.append(data[i][5])
    land_area_sqmi_2016.append(data[i][6])
    land_area_sqkm_2016.append(data[i][7])
    population_density_sqmi_2016.append(data[i][8])
    population_density_sqkm_2016.append(data[i][9])
    location.append(data[i][10])
    city_link.append(data[i][11])
    #demonyms.append(i)
    demonyms.append(get_demonym(data[i][11]))
    
df['Rank'] = rank
df['City'] = city
df['State'] = state
df['2018_Estimate'] = estimate_2018
df['2010_Cenius'] = census_2010
df['Change'] = change
df['2016_land_area_sqmi'] = data_cleansing(land_area_sqmi_2016, float)
df['2016_land_area_sqkm'] = data_cleansing(land_area_sqkm_2016, float)
df['2016_population_density_sqmi'] = data_cleansing(population_density_sqmi_2016, int)
df['2016_population_density_sqkm'] = data_cleansing(population_density_sqkm_2016, int)
df['Location'] = location
df['cit_link'] = city_link
df['Demonym'] = demonyms


df

,Rank,City,State,2018_Estimate,2010_Cenius,Change,2016_land_area_sqmi,2016_land_area_sqkm,2016_population_density_sqmi,2016_population_density_sqkm,Location,cit_link,Demonym
0,1,New York City[d],New York,"8,398,748","8,175,133",+2.74%,301.5,780.9,28317,10933,40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,https://en.wikipedia.org/wiki/New_York_City,New Yorker
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7,1213.9,8484,3276,34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,https://en.wikipedia.org/wiki/Los_Angeles,Angeleno
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3,588.7,11900,4600,41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,https://en.wikipedia.org/wiki/Chicago,Chicagoan
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5,1651.1,3613,1395,29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,https://en.wikipedia.org/wiki/Houston,Houstonian[1]
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6,1340.6,3120,1200,33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,"https://en.wikipedia.org/wiki/Phoenix,_Arizona",
5,6,Philadelphia[e],Pennsylvania,"1,584,138","1,526,006",+3.81%,134.2,347.6,11683,4511,40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...,https://en.wikipedia.org/wiki/Philadelphia,Philadelphian
6,7,San Antonio,Texas,"1,532,233","1,327,407",+15.43%,461.0,1194.0,3238,1250,29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W﻿...,https://en.wikipedia.org/wiki/San_Antonio,San Antonian
7,8,San Diego,California,"1,425,976","1,307,402",+9.07%,325.2,842.3,4325,1670,32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°...,https://en.wikipedia.org/wiki/San_Diego,San Diegan
8,9,Dallas,Texas,"1,345,047","1,197,816",+12.29%,340.9,882.9,3866,1493,32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W﻿...,https://en.wikipedia.org/wiki/Dallas,Dallasite
9,10,San Jose,California,"1,030,119","945,942",+8.90%,177.5,459.7,5777,2231,37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°...,"https://en.wikipedia.org/wiki/San_Jose,_Califo...",San Josean(s) Josefino/a(s)


In [ ]:
df

In [ ]:
# Converting resultant data frame to CSV file that is ready to be uploaded to a BigQuery table

In [ ]:
df.to_csv('us_city_information.csv', sep='\t', encoding='utf-8')